In [7]:
!python -V

Python 3.10.13


In [8]:
import pandas as pd

In [9]:
import pickle

In [10]:
import seaborn as sns
import matplotlib.pyplot as plt

In [11]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error as root_mean_squared_error

In [12]:

!pip install mlflow


!pip install mlflow[pipelines] 
!conda install -c conda-forge mlflow-pipelines

!pip install mlflow[extras]

!pip install mlflow-skinny


[notice] A new release of pip is available: 23.0.1 -> 24.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.1.1
[notice] To update, run: pip install --upgrade pip
Channels:
 - conda-forge
 - defaults
Platform: linux-64
Solving environment: warning  libmamba Added empty dependency for problem type SOLVER_RULE_UPDATE
failed

LibMambaUnsatisfiableError: Encountered problems while solving:
  - package mlflow-pipelines-1.28.0-py310h56eec8b_0 requires python >=3.10,<3.11.0a0, but none of the providers can be installed

Could not solve for environment specs
The following packages are incompatible
├─ mlflow-pipelines is installable with the potential options
│  ├─ mlflow-pipelines [1.28.0|1.29.0|1.30.0] would require
│  │  └─ python >=3.10,<3.11.0a0 , which can be installed;
│  ├─ mlflow-pipelines [1.28.0|1.29.0|1.30.0] would require
│  │  └─ python >=3.7,<3.8.0a0 , which can be installed;
│  ├─ mlflow-pipelines [1.28.0|1.29.0|1.30

In [38]:
import mlflow
 
 
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='/workspaces/mlops-zoomcamp/env1/02-experiment-tracking/mlruns/1', creation_time=1719836589746, experiment_id='1', last_update_time=1719836589746, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [14]:
import mlflow

In [15]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [16]:
df_train = read_dataframe("/workspaces/mlops-zoomcamp/env1/02-experiment-tracking/data/green_tripdata_2021-01.parquet")
df_val = read_dataframe("/workspaces/mlops-zoomcamp/env1/02-experiment-tracking/data/green_tripdata_2021-02.parquet")

In [17]:
len(df_train), len(df_val)

(73908, 61921)

In [18]:
df_train.dtypes

VendorID                          int64
lpep_pickup_datetime     datetime64[us]
lpep_dropoff_datetime    datetime64[us]
store_and_fwd_flag               object
RatecodeID                      float64
PULocationID                     object
DOLocationID                     object
passenger_count                 float64
trip_distance                   float64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
ehail_fee                        object
improvement_surcharge           float64
total_amount                    float64
payment_type                    float64
trip_type                       float64
congestion_surcharge            float64
duration                        float64
dtype: object

In [19]:
df_train['PU_DO'] = df_train['PULocationID'].astype(str) + '_' + df_train['DOLocationID'].astype(str)
df_val['PU_DO'] = df_val['PULocationID'].astype(str) + '_' + df_val['DOLocationID'].astype(str)

In [20]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [21]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [22]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

root_mean_squared_error(y_val, y_pred)

60.19766170466965

In [23]:
with open("/workspaces/mlops-zoomcamp/env1/02-experiment-tracking/models/lin_reg.bin", "wb") as f_out:
    pickle.dump((dv, lr), f_out)

In [24]:
# !pip install mlflow

In [37]:
import numpy as np
experiment_id="1"
mlflow.set_experiment(experiment_id=experiment_id)

with mlflow.start_run():

    mlflow.set_tag("developer", "Kasi")

    mlflow.log_param("train-data-path", "/workspaces/mlops-zoomcamp/env1/02-experiment-tracking/data/green_tripdata_2021-01.parquet")
    mlflow.log_param("valid-data-path", "/workspaces/mlops-zoomcamp/env1/02-experiment-tracking/data/green_tripdata_2021-02.parquet")

    alpha = 0.01
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = root_mean_squared_error(y_val, y_pred)
    mse = np.sqrt(rmse)
    print(mse,rmse)
    
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="/workspaces/mlops-zoomcamp/env1/02-experiment-tracking/models/lin_reg.bin", artifact_path="models_pickle")

11.167275941179728 124.70805194645156


In [26]:
import xgboost as xgb

In [27]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [28]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [29]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=10,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = root_mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [30]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=5,
    trials=Trials()
)

  0%|          | 0/5 [00:00<?, ?trial/s, best loss=?]

/workspaces/mlops-zoomcamp/env1/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [04:55:11] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:9.37481                          
[1]	validation-rmse:7.95703                          
[2]	validation-rmse:7.28890                          
[3]	validation-rmse:6.98202                          
[4]	validation-rmse:6.83557                          
[5]	validation-rmse:6.75119                          
[6]	validation-rmse:6.70691                          
[7]	validation-rmse:6.67993                          
[8]	validation-rmse:6.65945                          
[9]	validation-rmse:6.64682                          
 20%|██        | 1/5 [00:07<00:30,  7.56s/trial, best loss: 6.646819077677881]

/workspaces/mlops-zoomcamp/env1/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

/workspaces/mlops-zoomcamp/env1/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [04:55:18] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:11.33711                                                  
[1]	validation-rmse:10.58769                                                  
[2]	validation-rmse:9.94938                                                   
[3]	validation-rmse:9.40724                                                   
[4]	validation-rmse:8.95048                                                   
[5]	validation-rmse:8.56701                                                   
[6]	validation-rmse:8.24645                                                   
[7]	validation-rmse:7.97911                                                   
[8]	validation-rmse:7.75740                                                   
[9]	validation-rmse:7.57325                                                   
 40%|████      | 2/5 [00:15<00:22,  7.63s/trial, best loss: 6.646819077677881]

...

/workspaces/mlops-zoomcamp/env1/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

/workspaces/mlops-zoomcamp/env1/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [04:55:36] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:9.04938                                                  
[1]	validation-rmse:7.58968                                                  
[2]	validation-rmse:6.95939                                                  
[3]	validation-rmse:6.69297                                                  
[4]	validation-rmse:6.57318                                                  
[5]	validation-rmse:6.51120                                                  
[6]	validation-rmse:6.47384                                                  
[7]	validation-rmse:6.45221                                                  
[8]	validation-rmse:6.43833                                                  
[9]	validation-rmse:6.42933                                                  
 80%|████████  | 4/5 [00:49<00:14, 14.71s/trial, best loss: 6.429329932237456]

/workspaces/mlops-zoomcamp/env1/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

/workspaces/mlops-zoomcamp/env1/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [04:56:00] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:8.44077                                                   
[1]	validation-rmse:7.20525                                                   
[2]	validation-rmse:6.82494                                                   
[3]	validation-rmse:6.69718                                                   
[4]	validation-rmse:6.64720                                                   
[5]	validation-rmse:6.62652                                                   
[6]	validation-rmse:6.61784                                                   
[7]	validation-rmse:6.61198                                                   
[8]	validation-rmse:6.60793                                                   
[9]	validation-rmse:6.60513                                                   
100%|██████████| 5/5 [00:52<00:00, 10.58s/trial, best loss: 6.429329932237456]


/workspaces/mlops-zoomcamp/env1/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(



In [31]:
mlflow.xgboost.autolog(disable=True)

In [32]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 10,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=10,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = root_mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

/workspaces/mlops-zoomcamp/env1/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [04:56:04] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:11.45836
[1]	validation-rmse:10.79725
[2]	validation-rmse:10.22104
[3]	validation-rmse:9.71995
[4]	validation-rmse:9.28684
[5]	validation-rmse:8.91201
[6]	validation-rmse:8.59100
[7]	validation-rmse:8.31518
[8]	validation-rmse:8.07799
[9]	validation-rmse:7.87706


/workspaces/mlops-zoomcamp/env1/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/workspaces/mlops-zoomcamp/env1/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [04:56:06] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)


In [33]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "/workspaces/mlops-zoomcamp/env1/02-experiment-tracking/data/green_tripdata_2021-01.parquet")
        mlflow.log_param("valid-data-path", "/workspaces/mlops-zoomcamp/env1/02-experiment-tracking/data/green_tripdata_2021-02.parquet")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)
        

2024/07/02 04:56:14 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.
2024/07/02 04:56:14 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/workspaces/mlops-zoomcamp/env1/lib/python3.10/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils."
2024/07/02 04:56:14 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/workspaces/mlops-zoomcamp/env1/lib/python3.10/site-packages/

AssertionError: /usr/local/python/3.10.13/lib/python3.10/distutils/core.py